In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset

import numpy as np

import math

import time

import dataloader
import training_fun

import optuna

import joblib

import HydroErr

import csv

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SEQ_LENGTH = 365 * 2
TARGET_SEQ_LENGTH = 365
BASE_LENGTH = SEQ_LENGTH - TARGET_SEQ_LENGTH

FORCING_DIM = 3

N_CATCHMENTS = 559

EPOCHS = 500

# training hyperparameters
TRAIN_YEAR = 29
PATIENCE = 20

use_amp = True
compile_model = False

if compile_model:
    torch.set_float32_matmul_precision("high")

memory_saving = False
if memory_saving:
    storge_device = "cpu"
    computing_device = DEVICE
    VAL_STEPS = 500
else:
    storge_device = DEVICE
    computing_device = DEVICE

c:\Users\User\anaconda3\envs\pytorch-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [2]:
dtrain_val = dataloader.Forcing_Data(
    "data/camels_train.csv",
    record_length=3652,
    storge_device=storge_device,
    seq_length=SEQ_LENGTH,
    target_seq_length=TARGET_SEQ_LENGTH,
    base_length=BASE_LENGTH,
)

dtest = dataloader.Forcing_Data(
    "data/camels_val.csv",
    record_length=4383,
    storge_device=storge_device,
    seq_length=SEQ_LENGTH,
    target_seq_length=TARGET_SEQ_LENGTH,
    base_length=BASE_LENGTH,
)

In [3]:
fpath = "./data/selected_catchments_attributes.csv"
catchment_attributes_og = np.genfromtxt(fpath, delimiter=",", skip_header=1)

catchment_attributes_og = torch.from_numpy(catchment_attributes_og[:,1:]).to(computing_device).to(torch.float32) # deselect catchment id and move to GPU

In [4]:
study = joblib.load("data/base_lstm_decoder_study.pkl")

# Classes

In [5]:
class Encoder(nn.Module):
    def __init__(
        self,
        catchment_attributes,
        attribute_dim=27,
        encoder_fc_hidden_dims=[32, 16],
        latent_dim=8,
        p=0,
    ):
        super(Encoder, self).__init__()

        self.attribute_dim = attribute_dim
        self.catchment_attributes = catchment_attributes

        # LSTM to latent code
        self.encoder_fc_hidden_dims = encoder_fc_hidden_dims
        self.fc_layers = []
        self.p = p
        for i in range((len(self.encoder_fc_hidden_dims))):
            in_dim = (
                self.attribute_dim if i == 0 else self.encoder_fc_hidden_dims[i - 1]
            )
            out_dim = self.encoder_fc_hidden_dims[i]

            self.fc_layers += [nn.Linear(in_dim, out_dim)]
            self.fc_layers += [nn.ReLU()]
            self.fc_layers += [nn.Dropout(p=self.p)]

        self.latent_dim = latent_dim
        self.fc_layers += [nn.Linear(self.encoder_fc_hidden_dims[-1], self.latent_dim)]

        self.fc_layers = nn.Sequential(*self.fc_layers)

    def forward(self, inputs):
        out = self.fc_layers(inputs)
        return out

    def encode(self, selected_catchments):
        selected_attributes = self.catchment_attributes[selected_catchments, :]

        out = self.fc_layers(selected_attributes)
        return out


class TimeDistributed(nn.Module):
    # Reference: https://discuss.pytorch.org/t/any-pytorch-function-can-work-as-keras-timedistributed/1346/4

    def __init__(self, module, batch_first=False):
        super(TimeDistributed, self).__init__()
        self.module = module
        self.batch_first = batch_first

    def forward(self, x):

        if len(x.size()) <= 2:
            return self.module(x)

        # Squash samples and timesteps into a single axis
        x_reshape = x.contiguous().view(
            -1, x.size(-1)
        )  # (samples * timesteps, input_size)

        y = self.module(x_reshape)

        # We have to reshape Y
        if self.batch_first:
            y = y.contiguous().view(
                x.size(0), -1, y.size(-1)
            )  # (samples, timesteps, output_size)
        else:
            y = y.view(-1, x.size(1), y.size(-1))  # (timesteps, samples, output_size)

        return y


class LSTM_decoder(nn.Module):
    def __init__(
        self,
        latent_dim,
        feature_dim,
        lstm_hidden_dim,
        fc_hidden_dims,
        num_lstm_layers=1,
        output_dim=1,
        p=0.2,
    ):
        super(LSTM_decoder, self).__init__()

        self.latent_dim = latent_dim
        self.feature_dim = feature_dim
        self.lstm_hidden_dim = lstm_hidden_dim
        self.num_lstm_layers = num_lstm_layers

        self.lstm = nn.LSTM(
            self.feature_dim + self.latent_dim,
            self.lstm_hidden_dim,
            num_layers=self.num_lstm_layers,
            batch_first=True,
        )

        # LSTM to latent code
        self.fc_hidden_dims = fc_hidden_dims
        self.fc_layers = []
        self.p = p
        for i in range((len(self.fc_hidden_dims))):
            in_dim = self.lstm_hidden_dim if i == 0 else self.fc_hidden_dims[i - 1]
            out_dim = self.fc_hidden_dims[i]

            self.fc_layers += [nn.Linear(in_dim, out_dim)]
            self.fc_layers += [nn.ReLU()]
            self.fc_layers += [nn.Dropout(p=self.p)]

        self.output_dim = output_dim
        self.fc_layers += [nn.Linear(self.fc_hidden_dims[-1], self.output_dim)]

        self.fc_layers = TimeDistributed(
            nn.Sequential(*self.fc_layers), batch_first=True
        )

    def forward(self, inputs, base_length=365):
        out, (_, _) = self.lstm(inputs)
        out = self.fc_layers(out[:, base_length:, :])

        return out

    def decode(self, code, x):

        code = code.expand(x.shape[1], -1, -1).transpose(0, 1)

        x = torch.cat((code, x), 2)
        out = self.forward(x).squeeze()

        return out


class Model_builder:
    def __init__(self, catchment_attributes, forcing_dim=3, attribute_dim=27):
        self.forcing_dim = forcing_dim
        self.attribute_dim = attribute_dim
        self.catchment_attributes = catchment_attributes

    def define_model(self, trial):
        lstm_hidden_dim = trial.suggest_int("lstm_hidden_dim", 4, 256)
        n_lstm_layers = trial.suggest_int("n_lstm_layers", 1, 2)
        n_fc_layers = trial.suggest_int("n_fc_layers", 1, 3)
        latent_dim_power = trial.suggest_int("LATENT_DIM_power", 1, 4)
        latent_dim = 2**latent_dim_power

        fc_hidden_dims = []
        for i in range(n_fc_layers):
            fc_dim = trial.suggest_int(f"fc_dim{i}", 2, 32)
            fc_hidden_dims.append(fc_dim)

        decoder = LSTM_decoder(
            latent_dim=latent_dim,
            feature_dim=self.forcing_dim,
            lstm_hidden_dim=lstm_hidden_dim,
            fc_hidden_dims=fc_hidden_dims,
            num_lstm_layers=n_lstm_layers,
            output_dim=1,
            p=0,
        )

        # define encoder
        n_encoder_fc_layers = trial.suggest_int("n_encoder_fc_layers", 1, 3)
        encoder_fc_hidden_dims = []
        for i in range(n_encoder_fc_layers):
            encode_fc_dim = trial.suggest_int(f"encode_fc_dim{i}", 2, 32)
            encoder_fc_hidden_dims.append(encode_fc_dim)

        encoder = Encoder(
            attribute_dim=self.attribute_dim,
            encoder_fc_hidden_dims=encoder_fc_hidden_dims,
            latent_dim=latent_dim,
            catchment_attributes=self.catchment_attributes,
        )

        return encoder, decoder

# Functions

In [6]:
def get_optimal_epochs(study):
    
    stats = study.best_trials[0].intermediate_values
    epochs = min(stats, key=lambda k: stats[k]) + 1
    
    return epochs

In [7]:
def get_final_model(study, dataset, catchment_attributes, epoch_scale = 19/29): #19/39

    trial = study.best_trial

    # define model
    model_builder = Model_builder(
        forcing_dim=3, attribute_dim=27, catchment_attributes=catchment_attributes
    )

    encoder, decoder = model_builder.define_model(trial)
    encoder, decoder = encoder.to(computing_device), decoder.to(computing_device)

    if compile_model:
        # pytorch2.0 new feature, complile model for fast training
        encoder, decoder = torch.compile(encoder), torch.compile(decoder)

    # define optimizers
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=1e-3)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=1e-3)

    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

    # define batch size
    batch_size = 64
    
    # define optimal epochs
    epochs = round(get_optimal_epochs(study)*epoch_scale)

    # steps per epoch
    steps = round(N_CATCHMENTS * TRAIN_YEAR / batch_size)

        # train model
    for epoch in range(epochs):

        # for each epoch get_random_batch method generates a batch that contains one year data for each catchment
        # repeat TRAIN_YEAR times to finish an epoch
        decoder.train()
        encoder.train()
        
        for step in range(steps):

            decoder_optimizer.zero_grad()
            encoder_optimizer.zero_grad()

            # put the models into training mode
            decoder.train()
            encoder.train()

            # get training batch and pass to device
            (x_batch, y_batch, selected_catchments) = dataset.get_random_batch(
                batch_size
            )

            x_batch, y_batch, selected_catchments = (
                x_batch.to(computing_device),
                y_batch.to(computing_device),
                selected_catchments.to(computing_device),
            )
            
            # slice batch for training
            with torch.autocast(
                device_type="cuda", dtype=torch.float16, enabled=use_amp
            ):
                code = encoder.encode(selected_catchments)

                # pass through decoder
                out = decoder.decode(code, x_batch)

                # compute loss
                loss = training_fun.mse_loss_with_nans(out, y_batch)
            
            scaler.scale(loss).backward()
            scaler.step(encoder_optimizer)
            scaler.step(decoder_optimizer)
            scaler.update()

    return encoder, decoder

# Example
# encoder, decoder = get_final_model(study, dtrain_val, catchment_attributes_og)

# test_scores, _, _ = val_model(
#     encoder=encoder,
#     decoder=decoder,
#     dataset=dtest,
#     storge_device=storge_device,
#     val_metric=HydroErr.kge_2009
# )

In [8]:
def random_swap_columns(tensor, k):
    # Step 1: Check if k is valid
    if k > tensor.size(1):
        raise ValueError("k cannot be greater than the number of columns in the tensor")
    
    # Step 2: Randomly select k columns
    indices = torch.randperm(tensor.size(1))[:k]
    
    # Step 3: Shuffle each selected column independently
    for idx in indices:
        # Randomly permute the indices of the rows in this column
        permuted_indices = torch.randperm(tensor.size(0))
        # Apply this permutation to the column
        tensor[:, idx] = tensor[permuted_indices, idx]

    return tensor

In [9]:
def val_model(
    encoder,
    decoder,
    dataset,
    storge_device,
    val_metric
):

    x, y = dataset.get_val_batch()
    x, y = x.to(storge_device), y.to(storge_device)

    encoder.eval()
    decoder.eval()

    preds = torch.ones(size=y.shape, device=storge_device)

    n_catchments = y.shape[1]
    selected_catchments = torch.arange(n_catchments, device=storge_device)

    with torch.no_grad():
        code = encoder.encode(selected_catchments)
        for i in range(x.shape[0]):
            x_sub = x[i, :, :, :]
            preds[i, :, :] = decoder.decode(code, x_sub)
            
    # reshape to compute performance
    y_reshape = y.swapaxes(0,1).reshape(n_catchments,-1).detach().to("cpu").numpy()
    preds_reshape = preds.swapaxes(0,1).reshape(n_catchments,-1).detach().to("cpu").numpy()
    
    scores = np.ones(n_catchments)
    
    for i in range(n_catchments):
        scores[i] = val_metric(observed_array=y_reshape[i,:], simulated_array=preds_reshape[i,:])
    
    return scores, preds_reshape, y_reshape

# Wrapper

In [10]:
def corruption_wrapper(portio_corrupted, catchment_attributes_og):
    
    # prepare catchment attribute data
    if portio_corrupted == 0:
        catchment_attributes = catchment_attributes_og
    else:
        k = round(portio_corrupted*catchment_attributes_og.shape[1])
        catchment_attributes = random_swap_columns(catchment_attributes_og, k)

    # train model
    encoder, decoder = get_final_model(study, dtrain_val, catchment_attributes)
    
    # predict code of each catchment
    selected_catchments = torch.arange(N_CATCHMENTS, device=storge_device)
    
    with torch.no_grad():
        code = encoder.encode(selected_catchments)

    
    test_scores, _, _ = val_model(
        encoder=encoder,
        decoder=decoder,
        dataset=dtest,
        storge_device=storge_device,
        val_metric=HydroErr.kge_2009
    )

    return test_scores, code

In [11]:
params = np.arange(0, 1.1, 0.1)  # Parameters from 0 to 1 inclusive, step of 0.1

# Prepare a CSV file to store the results
with open('data/test_kges.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    # Write the header row
    writer.writerow(['Parameter', 'Trial', 'Results'])
    
    # Number of trials per parameter
    num_trials = 50
    
    # Run simulations and write results to the CSV file
    for param in params:
        for trial in range(num_trials):
            # Run the simulation with the current parameter
            test_kge, code = corruption_wrapper(param, catchment_attributes_og)
            
            # Write the results to the CSV file
            # Flatten the simulation output to write as a single row
            writer.writerow([param, trial] + test_kge.tolist())
            np.savetxt(X=code.cpu().numpy(), fname=f'data/code/{param}_{trial}.csv')
            
            # print(simulation_output)
            print(f'param={param}, trial={trial}')
            
            torch.cuda.empty_cache()

c:\Users\User\anaconda3\envs\pytorch-env\Lib\site-packages\HydroErr\HydroErr.py:6248: UserWarning: Row(s) [4018 4019 4020 4021 4022 4023 4024 4025 4026 4027 4028 4029 4030 4031
 4032 4033 4034 4035 4036 4037 4038 4039 4040 4041 4042 4043 4044 4045
 4046 4047 4048 4049 4050 4051 4052 4053 4054 4055 4056 4057 4058 4059
 4060 4061 4062 4063 4064 4065 4066 4067 4068 4069 4070 4071 4072 4073
 4074 4075 4076 4077 4078 4079 4080 4081 4082 4083 4084 4085 4086 4087
 4088 4089 4090 4091 4092 4093 4094 4095 4096 4097 4098 4099 4100 4101
 4102 4103 4104 4105 4106 4107 4108 4109 4110 4111 4112 4113 4114 4115
 4116 4117 4118 4119 4120 4121 4122 4123 4124 4125 4126 4127 4128 4129
 4130 4131 4132 4133 4134 4135 4136 4137 4138 4139 4140 4141 4142 4143
 4144 4145 4146 4147 4148 4149 4150 4151 4152 4153 4154 4155 4156 4157
 4158 4159 4160 4161 4162 4163 4164 4165 4166 4167 4168 4169 4170 4171
 4172 4173 4174 4175 4176 4177 4178 4179 4180 4181 4182 4183 4184 4185
 4186 4187 4188 4189 4190 4191 4192 4193 4

param=0.0, trial=0
param=0.0, trial=1
param=0.0, trial=2
param=0.0, trial=3


# Recycle

In [63]:
test_scores

array([ 7.16488175e-01,  6.86711528e-01,  6.98960070e-01,  8.44983789e-01,
        8.15278183e-01,  8.22822592e-01,  7.58290872e-01,  7.92015105e-01,
        8.27465270e-01,  8.27639020e-01,  7.94790988e-01,  7.75781070e-01,
        8.80504154e-01,  8.06206740e-01,  8.93793677e-01,  7.85213663e-01,
        8.86683339e-01,  8.37099592e-01,  7.19316528e-01,  8.09212245e-01,
        7.68115259e-01,  7.42876088e-01,  7.96899387e-01,  6.13493306e-01,
        7.99753061e-01,  5.67060632e-01,  7.77975144e-01,  1.24173033e-02,
        7.03777020e-01,  7.71613729e-01,  6.46190832e-01,  7.89936427e-01,
        7.46565802e-01,  8.37960782e-01,  8.07937540e-01,  8.75073454e-01,
        7.61155026e-01,  7.74332935e-01,  5.00300676e-01,  7.44882591e-01,
        7.73634013e-01,  6.03633988e-01,  6.42199953e-01,  7.57817867e-01,
        6.64121469e-01,  7.63302344e-01,  7.72743230e-01,  7.97975036e-01,
        7.35259037e-01,  7.64635634e-01,  7.73930326e-01,  7.83449536e-01,
        8.14099990e-01,  

In [20]:
dim = 0
idx = torch.randperm(catchment_attributes_og.shape[dim])

catchment_attributes = catchment_attributes_og[idx]

catchment_attributes_og, catchment_attributes

(tensor([[  3.1267,   1.9716,   0.6306,  ...,  16.2757,   0.0000, -14.7019],
         [  3.6081,   2.1193,   0.5874,  ...,  12.0376,   0.0000, -14.2138],
         [  3.2744,   2.0436,   0.6241,  ...,  14.7768,   0.0521, -14.4918],
         ...,
         [  6.6944,   2.1610,   0.3228,  ...,  15.7000,   0.0000, -14.8451],
         [  6.6700,   2.1729,   0.3258,  ...,  20.2302,   0.0000, -14.2961],
         [  4.5434,   2.2776,   0.5013,  ...,  24.3636,   0.0000, -13.5958]],
        device='cuda:0'),
 tensor([[  3.2613,   2.7735,   0.8504,  ...,  30.6903,   0.9980, -11.8017],
         [  2.5105,   2.8004,   1.1155,  ...,  35.2543,   0.0000, -13.9559],
         [  3.6174,   2.8047,   0.7753,  ...,  24.2387,   0.2822, -14.0789],
         ...,
         [  2.2289,   3.3455,   1.5010,  ...,  21.5914,   1.0000, -11.8000],
         [  1.9174,   3.3989,   1.7727,  ...,  20.3748,   0.9243, -11.8909],
         [  1.7660,   3.2502,   1.8404,  ...,  35.7589,   0.3191, -12.2767]],
        device='cuda

In [31]:
# random swap rows
idx = torch.randperm(catchment_attributes_og.shape[0])
catchment_attributes = catchment_attributes_og[idx]

# swap random selected columns
portion_selected = 0.2
k_selected_column = round(portion_selected*catchment_attributes_og.shape[1])



5

In [54]:
random_swap_columns(catchment_attributes_og, 27)

tensor([[  3.1531,   3.4349,   0.6225,  ...,  19.4322,   0.0916, -15.2000],
        [  2.0726,   2.9162,   0.5773,  ...,  45.1132,   0.0000, -15.2000],
        [  3.4225,   3.1343,   0.6114,  ...,  15.8930,   0.0000, -13.0532],
        ...,
        [  3.8148,   3.2012,   1.5632,  ...,  46.7212,   0.0000, -14.0443],
        [  7.2745,   2.5500,   0.8015,  ...,  14.0250,   0.0000, -10.9658],
        [  3.8417,   2.6528,   0.6289,  ...,  25.6986,   0.0000, -14.2526]],
       device='cuda:0')

In [26]:
k = 2
perm = torch.randperm(catchment_attributes_og.shape[1])
catchment_attributes_og[:,perm[:k]] = catchment_attributes[:,perm[:k]]

In [28]:
catchment_attributes_og[:,perm[:k]], catchment_attributes[:,perm[:k]]

(tensor([[9.9804e-01, 1.4564e+03],
         [0.0000e+00, 3.9981e+02],
         [2.8217e-01, 2.1494e+03],
         ...,
         [1.0000e+00, 4.3599e+02],
         [9.2428e-01, 1.9614e+03],
         [3.1907e-01, 6.1494e+02]], device='cuda:0'),
 tensor([[9.9804e-01, 1.4564e+03],
         [0.0000e+00, 3.9981e+02],
         [2.8217e-01, 2.1494e+03],
         ...,
         [1.0000e+00, 4.3599e+02],
         [9.2428e-01, 1.9614e+03],
         [3.1907e-01, 6.1494e+02]], device='cuda:0'))

In [21]:
catchment_attributes_og.shape[dim]

559